In [1]:
import os
import random

from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import imshow_norm, ZScaleInterval
from astropy.visualization.stretch import  SqrtStretch


import matplotlib.pyplot as plt

import pyvo.dal # Need this to prevent circular imports
import pyvo.auth.authsession
from pyvo.dal import SIA2Service
from pyvo.dal.adhoc import DatalinkResults

import requests


def _get_auth():
    s = requests.Session()

    token = os.environ["ACCESS_TOKEN"]
    s.headers["Authorization"] = "Bearer " + token
    auth = pyvo.auth.authsession.AuthSession()
    auth.credentials.set("lsst-token", s)

    auth.add_security_method_for_url("https://usdf-rsp-dev.slac.stanford.edu/api/datalink", "lsst-token")
    auth.add_security_method_for_url("https://usdf-rsp-dev.slac.stanford.edu/api/siav2", "lsst-token")
    auth.add_security_method_for_url("https://usdf-rsp-dev.slac.stanford.edu/api/siav2/query", "lsst-token")

    return auth

auth = _get_auth()

service = SIA2Service(
    "https://usdf-rsp-dev.slac.stanford.edu/api/siav2/",
    session=auth
)

def get_datalink_result(result):
    return DatalinkResults.from_result_url(result.getdataurl(),session=auth)

ImportError: cannot import name 'SIA2Service' from 'pyvo.dal' (/home/c/cbanek/env/lib/python3.11/site-packages/pyvo/dal/__init__.py)

In [ ]:
results = service.search(pos=(13.307, -72.517, 1.0))
results.to_table()

In [ ]:
# Extract the access URL from the result in the first row
result = random.choice(results)
f"Datalink link service url: {result.getdataurl()}"

In [ ]:
dl_result = get_datalink_result(result)
dl_result.to_table().show_in_notebook()

In [ ]:
# Full image of calexp - not a cutout
image_url = dl_result.getrecord(0).get('access_url')
image_url

In [ ]:
# Now let's download the image 
# filename = download_file(image_url)
# filename

In [ ]:
hdulist = fits.open(image_url)

for hdu in hdulist:
    print(hdu.name)

In [ ]:
# Let's plot the image and see what it looks like...
image = hdulist[1].data

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = imshow_norm(image, ax, origin='lower', interval=ZScaleInterval(), stretch=SqrtStretch(), cmap='gray')
fig.colorbar(im[0])